In [2]:
import pandas as pd
import numpy as np
import pandas_market_calendars as mcal

# for Robust location estimation
from statsmodels import robust

#### Load closing price, index composition and capitalization of all SX5E stocks

In [3]:
close_price = pd.read_csv("data/SX5E/close_mat.csv")
index_composition = pd.read_csv("data/SX5E/SX5E Index_compositionMat.csv")
capitalization = pd.read_csv("data/SX5E/capitalization_mat.csv")

In [4]:
display(close_price.head())

,Unnamed: 0,URW NA Equity,LINU GY Equity,LIN GY Equity,UN01 GY Equity,OSR GY Equity,AMS SQ Equity,VOW3 GY Equity,APAM NA Equity,UL NA Equity,...,INGA NA Equity,ENI IM Equity,RDA NA Equity,AVE FP Equity,EOAN GY Equity,DBK GY Equity,ALU FP Equity,ABI BB Equity,DTE GY Equity,ACA FP Equity
0,20191013,132.500000,NaN,177.550003,27.500000,38.430000,66.820000,138.955765,22.510000,NaN,...,9.838,13.836,NaN,NaN,8.84,6.797,NaN,83.940002,15.456,11.220
1,20191012,132.500000,NaN,177.550003,27.500000,38.430000,66.820000,138.955765,22.510000,NaN,...,9.838,13.836,NaN,NaN,8.84,6.797,NaN,83.940002,15.456,11.220
2,20191011,132.500000,NaN,177.550003,27.500000,38.430000,66.820000,138.955765,22.510000,NaN,...,9.838,13.836,NaN,NaN,8.84,6.797,NaN,83.940002,15.456,11.220
3,20191010,130.600006,NaN,174.449997,27.120001,38.770000,65.839996,132.451080,21.299999,NaN,...,9.467,13.682,NaN,NaN,8.68,6.464,NaN,83.589996,15.304,10.735
4,20191009,130.949997,NaN,169.350006,27.139999,39.049999,64.760002,129.061081,20.600000,NaN,...,9.064,13.600,NaN,NaN,8.82,6.294,NaN,84.290001,15.350,10.490


In [5]:
display(index_composition.head())

,Unnamed: 0,URW NA Equity,LINU GY Equity,LIN GY Equity,UN01 GY Equity,OSR GY Equity,AMS SQ Equity,VOW3 GY Equity,APAM NA Equity,UL NA Equity,...,INGA NA Equity,ENI IM Equity,RDA NA Equity,AVE FP Equity,EOAN GY Equity,DBK GY Equity,ALU FP Equity,ABI BB Equity,DTE GY Equity,ACA FP Equity
0,20191013,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,20191012,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,20191011,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,20191010,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,20191009,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [6]:
display(capitalization.head())

,Unnamed: 0,URW NA Equity,LINU GY Equity,LIN GY Equity,UN01 GY Equity,OSR GY Equity,AMS SQ Equity,VOW3 GY Equity,APAM NA Equity,UL NA Equity,...,INGA NA Equity,ENI IM Equity,RDA NA Equity,AVE FP Equity,EOAN GY Equity,DBK GY Equity,ALU FP Equity,ABI BB Equity,DTE GY Equity,ACA FP Equity
0,20191013,18334.214844,NaN,105009.804688,10030.963867,3713.159912,28817.357422,80811.640625,1884.000000,NaN,...,38334.421875,50282.589844,NaN,NaN,23349.259766,14047.857422,NaN,169495.171875,73593.101562,32366.208984
1,20191012,18334.214844,NaN,105009.804688,10030.963867,3713.159912,28817.357422,80811.640625,1884.000000,NaN,...,38334.421875,50282.589844,NaN,NaN,23349.259766,14047.857422,NaN,169495.171875,73593.101562,32366.208984
2,20191011,18334.214844,NaN,105009.804688,10030.963867,3713.159912,28817.357422,80811.640625,1884.000000,NaN,...,38334.421875,50282.589844,NaN,NaN,23349.259766,14047.857422,NaN,169495.171875,73593.101562,32366.208984
3,20191010,18071.308594,NaN,103377.218750,9902.877930,3746.080078,28394.712891,77128.078125,1782.729980,NaN,...,36888.792969,49722.925781,NaN,NaN,22926.648438,13359.621094,NaN,168788.421875,72869.359375,30967.132812
4,20191009,18119.738281,NaN,101301.351562,9939.473633,3760.209961,27928.945312,75223.796875,1724.140015,NaN,...,35318.476562,49424.921875,NaN,NaN,23296.431641,13008.270508,NaN,170201.906250,73088.390625,30260.384766


Now convert date column into datetime and add a column with the corresponding weekdays

In [7]:
def setup_tables(df: pd.DataFrame):
    X = df.copy()
    col_name = X.columns[0]
    X.rename(columns={col_name: "Date"}, inplace=True)
    X["Date"] = pd.to_datetime(X["Date"], format="%Y%m%d")
    
    def _function(x):
        return x.day_name()
    
    days = X["Date"].apply(_function)
    
    X.insert(1, column="Day_of_week", value=days)
    X.sort_values(by=["Date"], inplace=True)
    X.reset_index(drop=True, inplace=True)
    return X

In [8]:
close_price = setup_tables(close_price)
index_composition = setup_tables(index_composition)
capitalization = setup_tables(capitalization)

In [9]:
# To verify
display(close_price.head())
print(close_price.dtypes)

,Date,Day_of_week,URW NA Equity,LINU GY Equity,LIN GY Equity,UN01 GY Equity,OSR GY Equity,AMS SQ Equity,VOW3 GY Equity,APAM NA Equity,...,INGA NA Equity,ENI IM Equity,RDA NA Equity,AVE FP Equity,EOAN GY Equity,DBK GY Equity,ALU FP Equity,ABI BB Equity,DTE GY Equity,ACA FP Equity
0,2002-01-02,Wednesday,NaN,NaN,22.700001,NaN,NaN,NaN,29.078930,NaN,...,21.385405,13.68,54.400002,77.599998,15.954892,62.140289,17.982100,18.748993,19.200001,15.947296
1,2002-01-03,Thursday,NaN,NaN,23.400000,NaN,NaN,NaN,29.797352,NaN,...,21.746307,14.07,54.900002,77.599998,15.988493,62.450993,19.614300,18.442657,19.820000,15.974324
2,2002-01-04,Friday,NaN,NaN,23.549999,NaN,NaN,NaN,29.934193,NaN,...,21.515945,13.93,54.799999,76.250000,15.918491,64.004501,19.728201,18.317621,20.490000,15.938286
3,2002-01-05,Saturday,NaN,NaN,23.549999,NaN,NaN,NaN,29.934193,NaN,...,21.515945,13.93,54.799999,76.250000,15.918491,64.004501,19.728201,18.317621,20.490000,15.938286
4,2002-01-06,Sunday,NaN,NaN,23.549999,NaN,NaN,NaN,29.934193,NaN,...,21.515945,13.93,54.799999,76.250000,15.918491,64.004501,19.728201,18.317621,20.490000,15.938286


Date              datetime64[ns]
Day_of_week               object
URW NA Equity            float64
LINU GY Equity           float64
LIN GY Equity            float64
                       ...      
DBK GY Equity            float64
ALU FP Equity            float64
ABI BB Equity            float64
DTE GY Equity            float64
ACA FP Equity            float64
Length: 92, dtype: object


In [10]:
def select_time_slice(df: pd.DataFrame, start: int = 20020102, end: int = 20191013) -> pd.DataFrame:
    X = df.copy()
    start = pd.to_datetime(start, format="%Y%m%d")
    end = pd.to_datetime(end, format="%Y%m%d")
    X = X[X["Date"] <= end]
    X = X[X["Date"] >= start]
    return X

In [11]:
# Example from 2014-01-01 to 2018-12-31
X_close = select_time_slice(close_price, 20140101, 20181231)
X_index_comp = select_time_slice(index_composition, 20140101, 20181231)
X_capitalization = select_time_slice(capitalization, 20140101, 20181231)
#display(X_close.head())
#display(X_index_comp.head())
#display(X_capitalization.head())

Now let's sum over rows to find the columns (the stocks) that are part of the SX5E index for all the considered period. 

In [12]:
sum_index_series = X_index_comp.iloc[:, 2:].sum(axis=0)
print(sum_index_series)

URW NA Equity      210.0
LINU GY Equity      35.0
LIN GY Equity       64.0
UN01 GY Equity       1.0
OSR GY Equity        0.0
                   ...  
DBK GY Equity     1727.0
ALU FP Equity        0.0
ABI BB Equity     1826.0
DTE GY Equity     1826.0
ACA FP Equity        0.0
Length: 90, dtype: float64


In [13]:
max(sum_index_series)

1826.0

This means that the total number of days is 1826, we want to select only the stocks (the columns) whose sum over the rows for the considered period is equal to 1826.

In [14]:
full_time_stock = []

for stock in sum_index_series.index:
    if sum_index_series[stock] == 1826:
        full_time_stock.append(stock)

In [15]:
len(full_time_stock)

40

In [16]:
X_close = X_close[["Date", "Day_of_week"] + full_time_stock]
X_index_comp = X_index_comp[["Date", "Day_of_week"] + full_time_stock]
X_capitalization = X_capitalization[["Date", "Day_of_week"] + full_time_stock]

In [17]:
X_close

,Date,Day_of_week,VOW3 GY Equity,ENGI FP Equity,BAYN GY Equity,BNP FP Equity,PHIA NA Equity,ISP IM Equity,SAN FP Equity,MC FP Equity,...,GLE FP Equity,ITX SQ Equity,OR FP Equity,BBVA SQ Equity,ASML NA Equity,SAP GY Equity,INGA NA Equity,ENI IM Equity,ABI BB Equity,DTE GY Equity
4382,2014-01-01,Wednesday,175.652130,17.094999,100.328484,56.650002,24.998766,1.7940,76.700958,119.502663,...,42.220001,23.960001,127.699997,8.9480,68.040001,62.310001,10.100,17.490,77.260002,12.43
4383,2014-01-02,Thursday,172.511642,16.715000,98.458710,55.540001,25.125425,1.7830,74.811279,118.286003,...,41.520000,23.719999,125.599998,8.7530,67.760002,61.669998,10.080,17.320,75.610001,12.13
4384,2014-01-03,Friday,172.511642,16.745001,99.196777,55.680000,25.242702,1.8090,75.845627,118.060699,...,41.325001,23.780001,126.550003,8.7250,67.750000,61.709999,10.085,17.480,76.459999,12.27
4385,2014-01-04,Saturday,172.511642,16.745001,99.196777,55.680000,25.242702,1.8090,75.845627,118.060699,...,41.325001,23.780001,126.550003,8.7250,67.750000,61.709999,10.085,17.480,76.459999,12.27
4386,2014-01-05,Sunday,172.511642,16.745001,99.196777,55.680000,25.242702,1.8090,75.845627,118.060699,...,41.325001,23.780001,126.550003,8.7250,67.750000,61.709999,10.085,17.480,76.459999,12.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6203,2018-12-27,Thursday,118.254364,12.020000,59.160000,38.544998,28.432636,1.9090,73.359215,247.149994,...,27.215000,22.190001,195.600006,4.5450,132.619995,85.790001,9.191,13.520,57.349998,14.59
6204,2018-12-28,Friday,119.527763,12.275000,60.560001,39.375000,28.798540,1.9398,74.811279,252.750000,...,27.795000,22.389999,198.100006,4.6410,136.619995,86.930000,9.358,13.748,57.500000,14.82
6205,2018-12-29,Saturday,119.527763,12.275000,60.560001,39.375000,28.798540,1.9398,74.811279,252.750000,...,27.795000,22.389999,198.100006,4.6410,136.619995,86.930000,9.358,13.748,57.500000,14.82
6206,2018-12-30,Sunday,119.527763,12.275000,60.560001,39.375000,28.798540,1.9398,74.811279,252.750000,...,27.795000,22.389999,198.100006,4.6410,136.619995,86.930000,9.358,13.748,57.500000,14.82


Now we extract the calendar for ONLY the trading days for the considered period.

In [18]:
start_period = '2014-01-01'
end_period = '2018-12-31'

EU500 = mcal.get_calendar('EUREX')

calendar = EU500.schedule(start_date=start_period, end_date=end_period)

#And finally convert the schedule into a daterange that will be the dateindex of our forecasted series.
dates = mcal.date_range(calendar, frequency='1D') 

In [19]:
dates

DatetimeIndex(['2014-01-02 16:30:00+00:00', '2014-01-03 16:30:00+00:00',
               '2014-01-06 16:30:00+00:00', '2014-01-07 16:30:00+00:00',
               '2014-01-08 16:30:00+00:00', '2014-01-09 16:30:00+00:00',
               '2014-01-10 16:30:00+00:00', '2014-01-13 16:30:00+00:00',
               '2014-01-14 16:30:00+00:00', '2014-01-15 16:30:00+00:00',
               ...
               '2018-12-14 16:30:00+00:00', '2018-12-17 16:30:00+00:00',
               '2018-12-18 16:30:00+00:00', '2018-12-19 16:30:00+00:00',
               '2018-12-20 16:30:00+00:00', '2018-12-21 16:30:00+00:00',
               '2018-12-24 11:30:00+00:00', '2018-12-27 16:30:00+00:00',
               '2018-12-28 16:30:00+00:00', '2018-12-31 11:30:00+00:00'],
              dtype='datetime64[ns, UTC]', length=1272, freq=None)

In [20]:
# consider only year month day
dates = dates.strftime('%Y-%m-%d')

In [21]:
dates

Index(['2014-01-02', '2014-01-03', '2014-01-06', '2014-01-07', '2014-01-08',
       '2014-01-09', '2014-01-10', '2014-01-13', '2014-01-14', '2014-01-15',
       ...
       '2018-12-14', '2018-12-17', '2018-12-18', '2018-12-19', '2018-12-20',
       '2018-12-21', '2018-12-24', '2018-12-27', '2018-12-28', '2018-12-31'],
      dtype='object', length=1272)

In [22]:
# set date column as index

X_close.set_index('Date', drop=True, inplace=True)
X_close.head()

,Day_of_week,VOW3 GY Equity,ENGI FP Equity,BAYN GY Equity,BNP FP Equity,PHIA NA Equity,ISP IM Equity,SAN FP Equity,MC FP Equity,ENEL IM Equity,...,GLE FP Equity,ITX SQ Equity,OR FP Equity,BBVA SQ Equity,ASML NA Equity,SAP GY Equity,INGA NA Equity,ENI IM Equity,ABI BB Equity,DTE GY Equity
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,Wednesday,175.652130,17.094999,100.328484,56.650002,24.998766,1.794,76.700958,119.502663,3.174,...,42.220001,23.960001,127.699997,8.948,68.040001,62.310001,10.100,17.49,77.260002,12.43
2014-01-02,Thursday,172.511642,16.715000,98.458710,55.540001,25.125425,1.783,74.811279,118.286003,3.134,...,41.520000,23.719999,125.599998,8.753,67.760002,61.669998,10.080,17.32,75.610001,12.13
2014-01-03,Friday,172.511642,16.745001,99.196777,55.680000,25.242702,1.809,75.845627,118.060699,3.178,...,41.325001,23.780001,126.550003,8.725,67.750000,61.709999,10.085,17.48,76.459999,12.27
2014-01-04,Saturday,172.511642,16.745001,99.196777,55.680000,25.242702,1.809,75.845627,118.060699,3.178,...,41.325001,23.780001,126.550003,8.725,67.750000,61.709999,10.085,17.48,76.459999,12.27
2014-01-05,Sunday,172.511642,16.745001,99.196777,55.680000,25.242702,1.809,75.845627,118.060699,3.178,...,41.325001,23.780001,126.550003,8.725,67.750000,61.709999,10.085,17.48,76.459999,12.27


Now subselect only the rows of the dataframe that falls in the training days.

In [23]:
X_close = X_close.loc[dates]

In [24]:
# check the presence of any Saturday or Sunday

X_close['ABI BB Equity'].isin(['Saturday', 'Sunday']).sum()

0

### Test : Huber location estimator

In [46]:
# Define a function to calculate Huber location estimator
def huber_loc(series):
    return robust.scale.huber(series)[0]

# Apply the function to all rolling columns of the DataFrame (except days of week)
huber_estimates = X_close.drop(['Day_of_week'], axis=1).rolling(window=4).apply(huber_loc)

In [47]:
display(huber_estimates)

,VOW3 GY Equity,ENGI FP Equity,BAYN GY Equity,BNP FP Equity,PHIA NA Equity,ISP IM Equity,SAN FP Equity,MC FP Equity,ENEL IM Equity,CS FP Equity,...,GLE FP Equity,ITX SQ Equity,OR FP Equity,BBVA SQ Equity,ASML NA Equity,SAP GY Equity,INGA NA Equity,ENI IM Equity,ABI BB Equity,DTE GY Equity
2014-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-07,172.027664,16.68875,99.049162,56.090000,25.362324,1.8270,75.614389,116.810246,3.1810,19.87625,...,41.98625,23.7625,125.912500,8.904250,66.770000,61.722499,10.12025,17.5100,75.939999,12.21000
2014-01-08,171.952377,16.72250,99.061462,56.752500,25.503056,1.8635,75.641741,115.582325,3.2080,20.04500,...,42.77625,23.7650,125.450001,9.062500,66.254999,61.942499,10.23900,17.5225,76.279999,12.23375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21,124.836483,12.61000,61.857500,40.083751,29.354433,1.9653,75.798384,251.987499,5.1175,18.85150,...,28.43125,23.2975,201.975002,4.670625,137.969997,87.789999,9.65300,13.8410,60.389999,15.16250
2018-12-24,124.195480,12.51250,61.352500,39.642501,29.058895,1.9471,75.308559,250.274998,5.1610,18.80100,...,28.13250,22.8825,200.762501,4.639500,134.904999,87.457499,9.50700,13.8310,59.764999,15.13000
2018-12-27,122.164917,12.31875,60.735000,39.058750,28.691818,1.9196,74.550205,248.962498,5.1420,18.69800,...,27.71125,22.4625,198.887501,4.593875,132.814999,86.599998,9.33925,13.7235,58.764999,14.93375
2018-12-28,121.093710,12.27125,60.625000,39.052500,28.654289,1.9183,74.318968,249.624996,5.1160,18.72100,...,27.67750,22.3950,198.012501,4.590625,133.274998,86.389999,9.33950,13.7040,58.222500,14.85750


In [ ]:
### plot original series (for 1 stock) with location ts using different windows (pick the one with less variability)

### Test : Tukey biweight location estimator

Tukey biweight estimator:

$$ \hat{\mu} = \frac{\sum_{i=1}^N w_i x_i}{\sum_{i=1}^N w_i} $$ 

where:

$$ w_i =  $$

In [51]:
def tukey_biw_loc(series):
    median = np.median(series)
    MAD = np.median(np.abs(series - median))
    c = 4.685  # Tukey's constant for consistency with the standard deviation
    u = (series - median) / (c * MAD)
    tukey_weights = np.where(np.abs(u) <= 1, (1 - u**2)**2, 0)
    return np.sum(series * tukey_weights) / np.sum(tukey_weights)

# Apply the function to all rolling columns of the DataFrame (except days of week)
tukey_estimates = X_close.drop(['Day_of_week'], axis=1).rolling(window=4).apply(tukey_biw_loc)

In [52]:
tukey_estimates

,VOW3 GY Equity,ENGI FP Equity,BAYN GY Equity,BNP FP Equity,PHIA NA Equity,ISP IM Equity,SAN FP Equity,MC FP Equity,ENEL IM Equity,CS FP Equity,...,GLE FP Equity,ITX SQ Equity,OR FP Equity,BBVA SQ Equity,ASML NA Equity,SAP GY Equity,INGA NA Equity,ENI IM Equity,ABI BB Equity,DTE GY Equity
2014-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-07,172.368240,16.736427,98.994572,55.672662,25.349416,1.816647,75.875061,116.866570,3.181839,19.753800,...,41.530526,23.752005,125.874284,8.766855,66.817589,61.679189,10.056658,17.526310,75.989380,12.216977
2014-01-08,172.246680,16.765077,98.993589,56.663333,25.530574,1.860656,75.875061,115.471871,3.207225,20.036248,...,42.641339,23.756605,125.838427,9.064483,65.766724,61.852893,10.221749,17.526916,76.650263,12.236454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21,124.836482,12.598300,61.366147,40.063993,29.340526,1.965154,75.764952,252.242671,5.146534,18.864828,...,28.436166,23.300400,202.018489,4.669055,137.938456,87.760586,9.661683,13.807593,60.363668,15.113587
2018-12-24,123.468435,12.484124,61.408902,39.316168,28.815228,1.923133,75.534426,251.562391,5.164601,18.775831,...,27.947193,22.584234,201.212849,4.637186,133.442674,87.103017,9.390614,13.791334,59.585856,15.048333
2018-12-27,123.425715,12.374076,61.260000,39.073034,28.692140,1.911398,74.551050,248.746790,5.173333,18.735032,...,27.755841,22.460253,199.036069,4.591121,132.803169,86.277030,9.338323,13.787030,58.979515,15.038817
2018-12-28,121.216015,12.277441,61.101592,39.064360,28.652339,1.911398,74.320924,249.746892,5.116205,18.780291,...,27.696479,22.284112,197.881978,4.589745,132.529136,86.423987,9.338767,13.765333,57.998967,14.920329
